In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         Jan-99                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

## Test 1: Drop Status and Special Considerations column (noisy variables that could be distracting model)

In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', and 'STATUS' and 'SPECIAL_CONSIDERATIONS' to try to improve the model accuracy (data is poor quality)
application_df = application_df.drop(columns=["EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"], axis=1)
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION     INCOME_AMT  ASK_AMT  IS_SUCCESSFUL  
0   Association              0     5000              1  
1  Co-operative         Jan-99   108590              1  
2   Association              0     5000              0  
3         Trust    10000-24999     6692              1  
4         Trust  100000-499999   142590              1

In [8]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
unique_app_types = application_df.APPLICATION_TYPE.value_counts()

In [10]:
# Determine which values to replace if counts are less than 200
replace_application = list(unique_app_types[unique_app_types < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
       
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
unique_classification_types = application_df.CLASSIFICATION.value_counts()

In [12]:
# Determine which values to replace if counts are less than 1000
replace_class = list(unique_classification_types[unique_classification_types < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
# Generate our categorical variable lists
categorical_var = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[categorical_var]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(categorical_var)
encode_df.head()

C:\Users\z002cr1\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  \
0                     0.0                   1.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  \
0                  0.0                  0.0                  0.0   
1                  1.0                  0.0                  0.0   
2                  0.0                  0.0                  1.0   
3                  1.0                  0.0                  0.0   
4                  1.0                  0.0                  0.0   

   APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  APPLICATION_TYPE_T8  \
0                  0.0                  0.0                  0.0   
1                  0.0                  0.0                  0.0   
2                  0.0                  0.0                  0.0   
3                  0.0                  0.0                  0.0   
4                  0.0                  0.0                  0.0   

   AFFILIATION_CompanySponsored  ...  ORGANIZATION_Trust  INCOME_AMT_0  \
0                           0.0  ...                 0.0           1.0   
1                           0.0  ...                 0.0           0.0   
2                           1.0  ...                 0.0           1.0   
3                           1.0  ...                 1.0           0.0   
4                           0.0  ...                 1.0           0.0   

   INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  \
0                     0.0                       0.0                 0.0   
1                     0.0                       0.0                 0.0   
2                     0.0                       0.0                 0.0   
3                     1.0                       0.0                 0.0   
4                     0.0                       1.0                 0.0   

   INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  INCOME_AMT_50M+  \
0               0.0                     0.0              0.0   
1               0.0                     0.0              0.0   
2               0.0                     0.0              0.0   
3               0.0                     0.0              0.0   
4               0.0                     0.0              0.0   

   INCOME_AMT_5M-10M  INCOME_AMT_Jan-99  
0                0.0                0.0  
1                0.0                1.0  
2                0.0                0.0  
3                0.0                0.0  
4                0.0                0.0  

[5 rows x 39 columns]

In [15]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(categorical_var,1)
application_df.head()

C:\Users\z002cr1\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                     0.0                   1.0   
1   108590              1                     0.0                   0.0   
2     5000              0                     0.0                   0.0   
3     6692              1                     0.0                   0.0   
4   142590              1                     0.0                   0.0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                   0.0                  0.0                  0.0   
1                   0.0                  1.0                  0.0   
2                   0.0                  0.0                  0.0   
3                   0.0                  1.0                  0.0   
4                   0.0                  1.0                  0.0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                  0.0                  0.0                  0.0  ...   
1                  0.0                  0.0                  0.0  ...   
2                  1.0                  0.0                  0.0  ...   
3                  0.0                  0.0                  0.0  ...   
4                  0.0                  0.0                  0.0  ...   

   ORGANIZATION_Trust  INCOME_AMT_0  INCOME_AMT_10000-24999  \
0                 0.0           1.0                     0.0   
1                 0.0           0.0                     0.0   
2                 0.0           1.0                     0.0   
3                 1.0           0.0                     1.0   
4                 1.0           0.0                     0.0   

   INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  \
0                       0.0                 0.0               0.0   
1                       0.0                 0.0               0.0   
2                       0.0                 0.0               0.0   
3                       0.0                 0.0               0.0   
4                       1.0                 0.0               0.0   

   INCOME_AMT_25000-99999  INCOME_AMT_50M+  INCOME_AMT_5M-10M  \
0                     0.0              0.0                0.0   
1                     0.0              0.0                0.0   
2                     0.0              0.0                0.0   
3                     0.0              0.0                0.0   
4                     0.0              0.0                0.0   

   INCOME_AMT_Jan-99  
0                0.0  
1                1.0  
2                0.0  
3                0.0  
4                0.0  

[5 rows x 41 columns]

In [16]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback]) 

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5671 - accuracy: 0.7226
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7294
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7305
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7313
Epoch 5/50
781/804 [============================>.] - ETA: 0s - loss: 0.5489 - accuracy: 0.7323
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7336
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7343
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7329
Epoch 9/50
804/804 [==============================] - 1s 1ms/s

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7213 - 319ms/epoch - 1ms/step
Loss: 0.5601080656051636, Accuracy: 0.7212827801704407


## Test 2: Add an additional hidden layer and change the activation function from relu to sigmoid

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3280      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,031
Trainable params: 6,031
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback]) 

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6163 - accuracy: 0.6745
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5740 - accuracy: 0.7291
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5724 - accuracy: 0.7301
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5712 - accuracy: 0.7300
Epoch 5/50
802/804 [============================>.] - ETA: 0s - loss: 0.5683 - accuracy: 0.7314
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5684 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5632 - accuracy: 0.7316
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5585 - accuracy: 0.7314
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7326
Epoch 9/50
804/804 [==============================] - 1s 1ms/s

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7276 - 285ms/epoch - 1ms/step
Loss: 0.5555629730224609, Accuracy: 0.727580189704895


## Test 3: Test adding more neurons to a hidden layer and changing the number of epochs

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 120)               4920      
                                                                 
 dense_23 (Dense)            (None, 50)                6050      
                                                                 
 dense_24 (Dense)            (None, 20)                1020      
                                                                 
 dense_25 (Dense)            (None, 1)                 21        
                                                                 
Total params: 12,011
Trainable params: 12,011
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [52]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback]) 

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7381
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7369
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5368 - accuracy: 0.7380
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7388
Epoch 5/100
775/804 [===========================>..] - ETA: 0s - loss: 0.5362 - accuracy: 0.7395
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7389
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7392
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7388
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7390
Epoch 9/100
804/804 [==============================] -

764/804 [===========================>..] - ETA: 0s - loss: 0.5297 - accuracy: 0.7422
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5309 - accuracy: 0.7414
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.7419
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5308 - accuracy: 0.7417
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5308 - accuracy: 0.7420
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5305 - accuracy: 0.7423
Epoch 70/100
770/804 [===========================>..] - ETA: 0s - loss: 0.5313 - accuracy: 0.7422
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.7431
Epoch 71/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5306 - accuracy: 0.7428
Epoch 72/100
804/804 

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5793 - accuracy: 0.7250 - 226ms/epoch - 845us/step
Loss: 0.5793145298957825, Accuracy: 0.7250145673751831


In [54]:
# Export final model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")